# Microsoft Movie Maker Project
By Ferdinand Beaman
* Student pace: part time
* Scheduled project review date/time: May 3rd, 2023
* Instructor name: Mark Barbour
* Blog post URL: TBD


## Project Goal

This project is here to inform Microsoft of what factors will help their first (hypothetical) movie be a success. And by "success", I mean "generate the most profits".

### Data Source and Exploration
The data I used came primarily from [imdb.com](https://www.imdb.com/) and [the-numbers.com](https://www.the-numbers.com/). These websites have information about:
* titles, both domestic and international
* release dates
* cast and crew
* budgets
* Ticket sales, both domestic and international

In [1]:
#Importing the necessary libraries to find, analyze, and display the data.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import os
from glob import glob
from zipfile import ZipFile

In [2]:
#Grabbing the CSV data
csv_files = glob("./zippedData/*.csv.gz")

csv_files_dict = {}
for filename in csv_files:
        filename_cleaned = os.path.basename(filename).replace(
            ".csv", "").replace(".", "_")
        filename_df = pd.read_csv(filename, index_col=0)
        csv_files_dict[filename_cleaned] = filename_df

In [3]:
#Grabbing the data from imdb

file_name = "./ZippedData/im.db.zip"
with ZipFile(file_name, 'r') as zip:
    zip.printdir()
    zip.extractall()

#Extracting table names
con = sqlite3.connect("./im.db")
cur = con.cursor()

cur.execute("""SELECT name FROM sqlite_master  
  WHERE type='table';""")
cur.fetchall()

FileNotFoundError: [Errno 2] No such file or directory: './ZippedData/im.db.zip'

In [ ]:
#All the tables I might work from

#TheMovieDatabase
tmdb = csv_files_dict["tmdb_movies_gz"]

#The Numbers
tn = csv_files_dict["tn_movie_budgets_gz"]

#Box Office Mojo
bom = csv_files_dict["bom_movie_gross_gz"]

#And all the individual tables from IMDB
imdb_movie_basics = pd.DataFrame(data = 
                                 cur.execute("""SELECT * 
                                 FROM movie_basics"""))

imdb_akas = pd.DataFrame(data = 
                         cur.execute("""SELECT * 
                         FROM movie_akas"""))

imdb_known_for = pd.DataFrame(data = 
                              cur.execute("""SELECT * 
                              FROM known_for"""))

imdb_directors = pd.DataFrame(data = 
                              cur.execute("""SELECT * 
                              FROM directors"""))

imdb_writers = pd.DataFrame(data = 
                              cur.execute("""SELECT * 
                              FROM writers"""))

imdb_persons = pd.DataFrame(data = 
                              cur.execute("""SELECT * 
                              FROM persons"""))

Since my goal is to find what movies are profitable and not what movies are well-reviewed, I'm ignoring the ratings-related files.

# Merging all the data

Everything we need is spread out amongst a few different dataframes. So we're going to stick them all together and then cut out what we don't need. 

There are three general categories of information I want to look at: the finances, the genre, and who worked on it. Lastly, we should also stick with modern movies... whatever that means.


## Lining the numbers up

The data from "the-numbers" needs a little cleaning to turn it into something useful. But it has all the financial data that we're going to using as our measure of "success".

In [ ]:
tn.head()

In [ ]:
#Making a dedicated "year" column
tn["year"] = tn["release_date"].map(lambda x: x[-4:])

#Turning budget and gross into integers from strings
tn["budget"] = tn["production_budget"].map(
    lambda x: x.replace("$", "")).map(lambda x: x.replace(
    ",","")).map(lambda x: int(x))

tn["world_gross"] = tn["worldwide_gross"].map(
    lambda x: x.replace("$", "")).map(lambda x: x.replace(
    ",","")).map(lambda x: int(x))

#Dropping unecessary columns
tn.drop(["production_budget","worldwide_gross"
       , "domestic_gross", "release_date"], 1, inplace = True)

#Dropping any duplicate entries.
tn.drop_duplicates(inplace = True)

Now to get the first number I really want: profits! (This will be the difference between worldwide gross and budget)

In [ ]:
tn["profits"] = tn["world_gross"] - tn["budget"]
#And, let's just see what the most profitable movies of all time were
tn.sort_values("profits", ascending = False).head(15)

Awful lot of Marvel and other Disney products on there. Practically half of the top 15.

Now there's no easy way to parse how much of that has to do with Marvel's branding or how much of it has to do with Marvel's quality. But it's in the dataset nonetheless.


## Getting a sense of style

I bet that most people's first answer to "What kind of movie should you make?" is a genre. So lets get those analyzed.

How are the genres labeled?

In [ ]:
pd.read_sql("""SELECT genres
            FROM movie_basics""", con)

It turns out that each entry is a list, but each list has only one element which is just a *single string* with all of the genres written down all at once. Or it is NoneType object (like the penultimate entry above). They will each require attention to handle properly.

In [ ]:
#Isolating the genre column from IMDB, turning the rows into a list of lists
genre_column = (pd.read_sql("""SELECT genres
                            FROM movie_basics""", con).values.tolist())
#Transforming that into a set
all_genres = []
for batch in genre_column:
        if batch[0]: #Skips over the "None" entries
            genres = batch[0].split(",") #Unpacking the genres for each movie
            for genre in genres:
                if genre in all_genres:
                    continue
                else:
                    all_genres.append(genre)
all_genres

We're going to have to make each genre into its own column if we're going to get anywhere.

In [ ]:
#The "movie_basics" table will be one of our final three tables to merge.
movie_basics = pd.read_sql("""SELECT *
            FROM movie_basics""", con)

#Making genre columns, filling with 0s to start
for genre in all_genres:
    movie_basics[genre] = 0

#Filling in rows that have no genres
movie_basics["genres"].fillna(value = "?", inplace = True)

#Function to assign genre label
def is_genre(df,genre):
    df.loc[(df.genres.str.contains(genre)), genre]=1
    return df

#Looping through
for x in all_genres:
    is_genre(movie_basics,x)
    
movie_basics.head()

Mwah! Just to orient myself, let's see what the most common genres are.

In [ ]:
movie_basics[all_genres].sum().sort_values(ascending = False)

Wow! There was no way that I would have guessed that documentaries would be king, but I understand in retrospect.

## Asking for Direction
In my experience, a director makes or breaks a movie as much as any actor, maybe even moreso.

In [ ]:
#Making DF from the directors table
directors = pd.read_sql("""SELECT person_id, movie_id, primary_name
            FROM directors
            LEFT JOIN persons
            USING(person_id)
            """,con)

#There were duplicates, for some reason.
directors.drop_duplicates(inplace = True)
#And a couple Nas
directors.dropna(inplace = True)
directors.reset_index(inplace = True)
directors

For fun, I decided to see who are the most common directors.

In [ ]:
directors.value_counts("primary_name").head(10)

Well that's odd, maybe they directed a lot of foreign films...
*Checks Google*
...Nope! Omer Pasha directs music videos.

I'll have to make sure my finalists are movie directors in particular.

Come to think of it, I'm surprised to not see Speilberg in the top 10. What does the database say?

In [ ]:
directors.value_counts("primary_name")["Steven Spielberg"]

*NINE?* 
It was here that I had thought I did something very wrong. But, thankfully, it turns out that the IMDB data is only from 2010 forward.<br> Phew.

I suppose that's it for this step. Next, I'll join them all together before cutting out the smaller and older films.

# Finalizing the dataset

## Stitching it together
First, I think it would be easy to sew the director table onto "movie basics" since they share an obvious column: movie_id.

In [ ]:
movies_directors = movie_basics.merge(directors, on = "movie_id",
                                      how = "left")
movies_directors.drop("index", axis = 1, inplace = True)
movies_directors

In [ ]:
#Now to combine the movies/directors with the finacnes from TheNumbers.
tn.rename(columns = {"movie": "primary_title"}, inplace = True)
big_data = tn.merge(movies_directors, on = "primary_title", how = "inner")
big_data.drop_duplicates(inplace=True)

#Oddly, "start year" and "year" columns disagree.
#And 'start year' is often a later year for some reason
big_data.drop(["start_year", "original_title", "runtime_minutes"], 
              inplace = True, axis = 1)
big_data

## Cutting the chaff
All of the movies I want are going to be less than 10 years old. However, I'm going to have to remove all the entries from 2020 and 2021 due to the pandemic.

Pre-pandemic, the average movie released in theaters made 12,487,209 in the US, so I think it'll be a fair starting point to say that in order for a movie to be "relevant" to my project, it has to have made over half of that.

In [ ]:
#For some reason the year entries were strings.
big_data.year = big_data.year.astype(int)

#Limiting to relevant times and finances
trim_data = big_data[(big_data["year"] > 2013) & (big_data["year"] < 2020) &
                     (big_data["world_gross"] > 6000000)]

trim_data

There are actually a lot of duplicates in this dataset. Why? Because movies with multiple directors are showing up as multiple entries.

I can foresee this being useful, so we'll keep this dataset with multiple directors as well as one where the duplications are cut out.

In [ ]:
trim_no_d = trim_data.drop_duplicates(subset = ["year", "primary_title"])
trim_no_d

In [ ]:
#One last thing to check.
#trim_no_d.primary_name.value_counts()["Omer Pasha"]

Good.

# Measurements

## Picking the Medium
This part should be straightforward. Animation or Live Action?

In [ ]:
#Getting the mean and median profits for animated and live action movies. 
live_mean_profits = trim_no_d[trim_no_d["Animation"] == 0]["profits"].mean()
ani_mean_profits = trim_no_d[trim_no_d["Animation"] == 1]["profits"].mean()

live_median_profits = trim_no_d[trim_no_d["Animation"] == 0
                               ]["profits"].median()
ani_median_profits = trim_no_d[trim_no_d["Animation"] == 1
                              ]["profits"].median()

In [ ]:
from matplotlib import ticker

#Preparing the formatting of my graphs
plt.rc('font', size=11)          # controls default text sizes
plt.rc('axes', titlesize=11)     # fontsize of the axes title
plt.rc('axes', labelsize=12)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
plt.rc('ytick', labelsize=11)    # fontsize of the tick labels
plt.rc('legend', fontsize=11)    # legend fontsize
plt.rc('figure', titlesize=14)  # fontsize of the figure title

fig, ax = plt.subplots(figsize=(7, 6))

plt.xlabel("Style", fontsize=13)
plt.ylabel("Profits", fontsize=14)
plt.title("Animation vs Live Action (Median Profits)", fontsize=14)

#Formatting the y-axis to be in dollars
ax.yaxis.set_major_locator(ticker.MultipleLocator(40000000.00))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(10000000))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('${x:,.0f}'))

ax = plt.bar(["Live_Action_Profits", "Animated_Profits"], 
        [live_median_profits, ani_median_profits])

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))

plt.xlabel("Style", fontsize=13)
plt.ylabel("Profits", fontsize=14)
plt.title("Animation vs Live Action (Mean Profits)", fontsize=14)

ax.yaxis.set_major_locator(ticker.MultipleLocator(40000000.00))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(10000000))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('${x:,.0f}'))

ax = plt.bar(["Live_Action_Profits", "Animated_Profits"], 
        [live_mean_profits, ani_mean_profits])

Well that's not even close! Animated movies are almost three times as profitable by mean, and nearly *quintuple* be median! One suggestion down, two to go.

Just how common are animated movies, anyway?

In [ ]:
print("There are", len(trim_no_d[trim_no_d["Animation"] == 1]),
      "animated movies")
print("There are", len(trim_no_d[trim_no_d["Animation"] == 0]),
      "live action movies")
print("That's",(len(trim_no_d[trim_no_d["Animation"] == 1]))/(
    len(trim_no_d[trim_no_d["Animation"] == 1])+
    len(trim_no_d[trim_no_d["Animation"] == 0])), "percent")

Okay, so that's a fair enough sample.

## Picking the Director

I'll get a look at the most profitable directors by both mean and median, then choose from among those the ones with the largest catalogs.

In [ ]:
#Profits Mean, top 15
trim_data.groupby("primary_name").mean().sort_values(
    "profits", ascending = False).head(15)

In [ ]:
#Profits Median
trim_data.groupby("primary_name").median().sort_values(
    "profits", ascending = False).head(15)

Clearly there's a bunch of overlap, and I am going to take just the directors whose names appear in both lists.

In [ ]:
#Isolating just the top fifteen directors by each metric

top_median_directors = trim_data.groupby("primary_name").median().sort_values(
    "profits", ascending = False).head(15).index

top_mean_directors = trim_data.groupby("primary_name").mean().sort_values(
    "profits", ascending = False).head(15).index

In [ ]:
#Creating the set of directors from both groups
top_directors = [name for name in top_median_directors 
                 if name in top_mean_directors]
top_directors

In [ ]:
len(top_directors)

Everyone was in both lists. Unexpected. That probably means a lot of people only have 1 qualified movie to their name.

In [ ]:
#making dfs that are grouped by mean and median
interim_mean = trim_data.groupby("primary_name").mean()
interim_median = trim_data.groupby("primary_name").median()

#selecting only the top 15 directors
top_director_interim_mean = interim_mean[
    interim_mean.index.isin(top_directors)]
top_director_interim_median = interim_median[
    interim_median.index.isin(top_directors)]

#sorting and isolating the profits column
d_means = top_director_interim_mean.sort_values(
    "profits", ascending = False)["profits"]
d_medians = top_director_interim_median.sort_values(
    "profits", ascending = False)["profits"]

In [ ]:
#Graphing the best of those results
fig, ax = plt.subplots(nrows =1, figsize = (11,7))

plt.ylabel("Profits", fontsize=14)

ax.yaxis.set_major_locator(ticker.MultipleLocator(250000000))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(50000000))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('${x:,.0f}'))

ax.bar(d_means.head(9).index, d_means.head(9))

ax.set_title("Mean Profit by Director", fontsize = 13)

ax.tick_params(axis = "x", labelrotation = 50)

In [ ]:
#Graphing those
fig, ax = plt.subplots(nrows =1, figsize = (11,7))

plt.ylabel("Profits", fontsize=14)

ax.yaxis.set_major_locator(ticker.MultipleLocator(250000000))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(50000000))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('${x:,.0f}'))

ax.bar(d_medians.head(10).index, d_medians.head(10))

ax.set_title("Median Profit by Director", fontsize = 13)

ax.tick_params(axis = "x", labelrotation = 50)

There really doesn't seem to be that big of a difference the further down the list we go. Surprisingly, the Russos (of Avengers: Endgame fame) fall out of the top 10 using the median.

How many movies these directors have made?

In [ ]:
for name in top_directors:
    print(name, trim_data.primary_name.tolist().count(name))

Ah, so the reason why the median scores are what they are is because so many of these filmmakers have only one qualifying flick. Even with this many candidates, only the Russos stand out with three relevant movies in the last 10 years. Even though they fell out of the top 10 by median, they were still able to even qualify for this list in the first place despite their second and third place movies being less impressive.

As for the other directors who had multiple films that qualified:<br>
James Wan directed The Conjuring and Aquaman<br>
Kyle Balda directed two of the correctly-maligned Minions movies<br>
and Pierre Coffin... also directed two Minions movies.

Colin Treverrow was the leader on both lists with one movie. Luckily for us, that film *was* both an Adventure and Sci-Fi movie, making him a very defensible choice.

All in all, I'd still give it to the Russo Brothers, because they've done it more consistently. But it's probably debatable. It may be impossible to know how much of their success just has to do with the momentum Marvel supplies just by brand recognition.

## Picking the genre
I'm going to make a dataframe that's just made up of the profits (mean and median) for each genre.

In [ ]:
#Preparing lists for my DF's columns
g_net_mean = []
g_net_median = []
g_count = []
#Filling those lists
all_genres.remove("Animation")
for genre in all_genres:
        g_net_mean.append(trim_no_d[trim_no_d[genre] == 1]["profits"].mean())
        g_net_median.append(
            trim_no_d[trim_no_d[genre] == 1]["profits"].median())
        g_count.append(len(trim_no_d[trim_no_d[genre] ==1]))

In [ ]:
#Birthing the DataFrame
df_genres = pd.DataFrame()
df_genres["genre"] = all_genres
df_genres["count"] = g_count
df_genres["net_mean"] = g_net_mean
df_genres["net_median"] = g_net_median
df_genres.head()

Now we isolate the sorted data for the profit columns.

In [ ]:
g_means = df_genres.sort_values("net_mean", ascending = False).head(10)
g_medians = df_genres.sort_values("net_median", ascending = False).head(10)

In [ ]:
g_means

In [ ]:
g_medians

Musicals take the gold medal?
Well, with only 2 representatives, I think it's fair to say that it's not exactly a tried and true formula for success. 

So, we'll take a closer look at the silver and bronze finalists instead: Sci-Fi and Adventure.

We can first make sure that outliers aren't driving the data, and we can also see if these genres are buoyed by data from the early 2010s.

In [ ]:
#Making a box blot of just the top genres
genre_cols = ["Adventure", "Sci-Fi", "Action",
              "Fantasy", "Family", "Comedy", "Thriller"]
boxplot_boxes = []
for genre in genre_cols:
    boxplot_boxes.append(trim_no_d[trim_no_d[genre] == 1]["profits"])
plt.boxplot(boxplot_boxes, vert = False, showfliers = False, labels =
            genre_cols)
plt.axvline();

While a little over 20% of our sample of Sci-Fi and Adventure movies lose money, it seems like many of the genres share this fate. And this boxplot has its outliers filtered out, so the big blockbusters are not solely responsible for these two pulling ahead.

In [ ]:
#Grouped bar graph, using Action as the control
fig, ax = plt.subplots()

ax.yaxis.set_major_locator(ticker.MultipleLocator(100000000))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(50000000))
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('${x:,.0f}'))

x = np.arange(6)
y1 = trim_no_d[(trim_no_d["Sci-Fi"] == 1)].groupby('year').mean()["profits"]
y2 = trim_no_d[(trim_no_d["Adventure"] == 1)].groupby(
    'year').mean()["profits"]
y3 = trim_no_d[(trim_no_d["Action"] == 1)].groupby('year').mean()["profits"]
width = 0.2
  
plt.bar(x-0.2, y1, width, color='mediumslateblue')
plt.bar(x, y2, width, color='indianred')
plt.bar(x+0.2, y3, width, color='mediumseagreen')
plt.xticks(x, [2014, 2015, 2016, 2017, 2018, 2019])
plt.xlabel("Years", fontsize = 12)
plt.ylabel("Profits", fontsize = 14)
plt.legend(["Sci-Fi", "Adventure", "Action"])
plt.title("Mean Profits of Genres by Year", fontsize = 14)
plt.show()

In [ ]:
#Why check by eye when we have math?
#Getting the Correlation Coefficient
np.corrcoef(y1, [2014, 2015, 2016, 2017, 2018, 2019])

Seems good to me. Let's just graph the overall means and medians now.

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols = 2, figsize = (15, 4))

#ax1.yaxis.set_major_locator(ticker.MultipleLocator(500000000))
#ax1.yaxis.set_minor_locator(ticker.MultipleLocator(250000000))
#ax1.yaxis.set_major_formatter(ticker.StrMethodFormatter('${x:,.0f}'))

#ax2.yaxis.set_major_locator(ticker.MultipleLocator(500000000))
#ax2.yaxis.set_minor_locator(ticker.MultipleLocator(250000000))
#ax2.yaxis.set_major_formatter(ticker.StrMethodFormatter('${x:,.0f}'))

#The bars will be colored based on whether the sample is high or low
colors1 = []
colors2 = []

for n in g_means["count"]:
    if n >= 30:
        colors1.append("mediumslateblue") #blue if 30 or higher
    else:
        colors1.append("indianred") #red if less than 30
        
for n in g_medians["count"]:
    if n >= 30:
        colors2.append("mediumslateblue")
    else:
        colors2.append("indianred")

genre_mean = g_means["net_mean"]
genre_median = g_medians["net_median"]

ax1.set_ylabel("Hundreds of Millions", fontsize = 11)
ax2.set_ylabel("Hundreds of Millions")

ax1.set_title("Mean Profit by Genre", fontsize = 14)
ax2.set_title("Median Profit by Genre")

ax1.tick_params(axis = "x", labelrotation = 50)
ax2.tick_params(axis = "x", labelrotation = 50)

#Creating the legend with its colors
import matplotlib.lines as mlines
low = mlines.Line2D([], [], color='indianred', 
                      marker='D', ls='', label='Num Movies in Sample < 30')
high = mlines.Line2D([], [], color='mediumslateblue', 
                     marker='D', ls='', label='Num Movies in Sample >= 30')
ax1.legend(handles=[low, high])
ax2.legend(handles=[low, high])

ax1.bar(g_means["genre"], genre_mean, color = colors1)
ax2.bar(g_medians["genre"], genre_median, color = colors2)

# So what do we conclude?
Putting it all together, I'm actually very pleased with the results.

Action and comedy are "ok", but I think that it would be wise to separate this project from a Marvel style movie. I'd suggest focusing on the two core genres to further make this film distinct from the competition.

So Microsoft could set itself apart with an *Animated Sci-Fi Adventure directed by the Russo Brothers*, perhaps with a more serious edge. If they turn it down, get Colin Treverrow. Failing that, James Wan will do.

I'd watch that.

## Limitations, Recommendations, Next Steps
It's important to note that, again, the movie business was gutted during the two Covid-stricken years. In fact, it's still hasn't fully rebounded. So data about current movie trends might be out of date, and releasing a movie very soon might not even be the best idea (depending on how much longer the recovery will take.)


Probably the most obvious places we didn't look in the data would be at both actors and writers. However, slightly less intuitively important might be to see if the release date (or even the release day of the week) matters. It may be worth looking at each of those, although it is probably hard to determine what actors to use without knowing what the roles and tone of the final product will be.